# Notebook: Entrenamiento y Prueba de Modelos de Regresión

Juan David Garcia Zapata

### Objetivo
En este notebook, aprenderás a entrenar y evaluar modelos de regresión usando la base de datos de Boston, calcular el Population Stability Index (PSI) y aplicar validación cruzada para obtener una estimación más robusta del rendimiento del modelo.

### 1. Cargar y explorar la base de datos:

In [ ]:
import pandas as pd
from sklearn.datasets import fetch_california_housing

# Cargar la base de datos de California
california = fetch_california_housing()
data = pd.DataFrame(california.data, columns=california.feature_names)
data['MEDV'] = california.target

# Mostrar las primeras filas del DataFrame
data.head()


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MEDV
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422


In [ ]:
print(data.describe())

             MedInc      HouseAge      AveRooms     AveBedrms    Population  \
count  20640.000000  20640.000000  20640.000000  20640.000000  20640.000000   
mean       3.870671     28.639486      5.429000      1.096675   1425.476744   
std        1.899822     12.585558      2.474173      0.473911   1132.462122   
min        0.499900      1.000000      0.846154      0.333333      3.000000   
25%        2.563400     18.000000      4.440716      1.006079    787.000000   
50%        3.534800     29.000000      5.229129      1.048780   1166.000000   
75%        4.743250     37.000000      6.052381      1.099526   1725.000000   
max       15.000100     52.000000    141.909091     34.066667  35682.000000   

           AveOccup      Latitude     Longitude          MEDV  
count  20640.000000  20640.000000  20640.000000  20640.000000  
mean       3.070655     35.631861   -119.569704      2.068558  
std       10.386050      2.135952      2.003532      1.153956  
min        0.692308     32.54000

#### Descripción de la base de datos:
El conjunto de datos de California housing contiene información sobre varias características de viviendas en California.


longitude y latitude: Pueden usarse para analizar la ubicación geográfica y su influencia en el valor de las viviendas. La proximidad a ciertas áreas puede afectar significativamente el valor.

housing_median_age: La antigüedad de las viviendas puede estar relacionada con el valor; por ejemplo, las viviendas más nuevas suelen tener un valor más alto.

total_rooms y total_bedrooms: Indican el tamaño de las viviendas y pueden influir en el valor, ya que más habitaciones o dormitorios suelen correlacionarse con un valor mayor.

population y households: Ayudan a entender la densidad poblacional y la distribución de las viviendas en el área.

median_income: Es un predictor clave del valor de la vivienda. Un ingreso medio más alto en un área suele correlacionarse con un valor de vivienda más alto.

median_house_value: Es la variable objetivo en el modelo de regresión, que intentamos predecir en función de las otras características.

#### Pregunta: ¿Cuál es la variable objetivo?

La variable objetivo en este conjunto de datos es **`MEDV`** (Median House Value), que representa el valor medio de las viviendas en cientos de miles de dólares. Esta es la variable que se intenta predecir en un modelo de regresión utilizando las otras características disponibles en el conjunto de datos.

## 2. Preprocesamiento de datos:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Dividir los datos en variables independientes (X) y variable dependiente (y)
X = data.drop('MEDV', axis=1)
y = data['MEDV']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalizar las características
scaler = StandardScaler()
# Aplicar escalamiento Min-Max
#scaler = MinMaxScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### Ejercicio: ¿Por qué es importante normalizar las características en un modelo de regresión? ¿Qué pasaría si no lo hicieras? ¿Qué ventajas ofrece el escalamiento Min-Max en comparación con otras técnicas de normalización? ¿hay diferencias en el desempeño del modelo al usar una u otra?


Normalizar las características en un modelo de regresión es crucial para asegurar estabilidad numérica y comparabilidad, especialmente en algoritmos sensibles a la escala como la regresión lineal. Sin normalización, características con diferentes escalas pueden desbalancear el modelo, afectando su rendimiento y la interpretación de los coeficientes. El escalamiento Min-Max es útil cuando se necesita que las características estén en un rango específico, siendo especialmente beneficioso para modelos basados en distancias, mientras que la normalización estándar (`StandardScaler`) es preferible en modelos lineales para estabilizar la convergencia.

## 3. Entrenar un Modelo de Regresión Lineal:

In [ ]:
from sklearn.linear_model import LinearRegression

# Crear y entrenar el modelo
modelo = LinearRegression()
modelo.fit(X_train_scaled, y_train)

# Realizar predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test_scaled)


#### Ejercicio: ¿Cómo se interpretan los coeficientes del modelo de regresión lineal? ¿Qué información proporcionan sobre la relación entre las características y la variable objetivo?

Los coeficientes del modelo de regresión lineal indican cómo cada característica influye en la variable objetivo (`MEDV`), con valores positivos indicando una relación directa y valores negativos una relación inversa. Por ejemplo, un mayor ingreso medio en la zona se asocia con un aumento en el valor de las viviendas, mientras que una mayor población o estar más al oeste disminuye el valor. Sin embargo, dado que los datos fueron normalizados antes de entrenar el modelo, la magnitud de los coeficientes es menos intuitiva, y la interpretabilidad directa de los cambios en las características puede ser cuestionable.

In [ ]:
inter = modelo.intercept_
print(f"Intercepto: {inter}")
coeficient = modelo.coef_
print("Coeficientes")
for i, col in enumerate(X.columns):
  print(f" {col}: {coeficient[i]}")

Intercepto: 2.0719469373788777
Coeficientes
 MedInc: 0.8543830309268546
 HouseAge: 0.12254623807840737
 AveRooms: -0.2944101344732999
 AveBedrms: 0.3392594905944844
 Population: -0.0023077231458302765
 AveOccup: -0.04082910308508747
 Latitude: -0.896928876638665
 Longitude: -0.8698417752417177


## 4. Evaluar el modelo:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

# Calcular MAE y MAPE
mae = mean_absolute_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100

print(f"MAE: {mae:.2f}")
print(f"MAPE: {mape:.2f}%")


MAE: 0.53
MAPE: 31.95%


#### Ejercicio: ¿Qué indican los valores de MAE y MAPE sobre el rendimiento del modelo? ¿Qué tan buenos son estos valores en el contexto del conjunto de datos de California?

Los valores de MAE (0.53) y MAPE (31.95%) indican que el modelo de regresión lineal tiene un rendimiento moderado en el conjunto de datos de California Housing, con un error medio absoluto de $530 y un error porcentual medio del 31.95%. Aunque el MAE sugiere una precisión razonable, el MAPE revela que el modelo se desvía un 31.95% en promedio de los valores reales, lo que indica que hay margen para mejorar la precisión del modelo.

## 5. Calcular el Population Stability Index (PSI)

In [ ]:
import numpy as np

def calculate_psi(expected, actual, bins=10):
    """Calcula el Population Stability Index (PSI)."""
    # Crear bins para la característica
    bins = np.linspace(min(expected.min(), actual.min()), max(expected.max(), actual.max()), bins + 1)
    # Contar las observaciones en cada bin para los datos esperados y actuales
    expected_counts, _ = np.histogram(expected, bins=bins)
    actual_counts, _ = np.histogram(actual, bins=bins)

    # Convertir las cuentas a proporciones
    expected_prop = expected_counts / expected_counts.sum()
    actual_prop = actual_counts / actual_counts.sum()

    # Manejar división por cero
    expected_prop = np.clip(expected_prop, 1e-10, 1)
    actual_prop = np.clip(actual_prop, 1e-10, 1)

    # Calcular PSI
    psi = np.sum((actual_prop - expected_prop) * np.log(actual_prop / expected_prop))

    return psi

# Calcular PSI para cada característica
psi_values = {}
for feature in X.columns:
    psi = calculate_psi(X_train[feature], X_test[feature])
    psi_values[feature] = psi

# Mostrar PSI para cada característica
psi_df = pd.DataFrame(list(psi_values.items()), columns=['Feature', 'PSI'])
psi_df.sort_values(by='PSI', ascending=False, inplace=True)
print(psi_df)


      Feature       PSI
3   AveBedrms  0.008715
2    AveRooms  0.007694
5    AveOccup  0.003309
6    Latitude  0.003242
0      MedInc  0.002595
7   Longitude  0.002327
4  Population  0.002320
1    HouseAge  0.001218


#### ¿Qué indica el valor del PSI para cada característica? ¿Cómo interpretarías el PSI en términos de la estabilidad de las características entre los conjuntos de entrenamiento y prueba?

El Population Stability Index (PSI) mide la estabilidad o el cambio en la distribución de una característica entre dos conjuntos de datos, como el conjunto de entrenamiento y el conjunto de prueba. Un PSI bajo generalmente indica que la distribución de una característica ha cambiado poco entre los conjuntos de entrenamiento y prueba, lo que sugiere estabilidad. Por otro lado, un PSI alto indicaría un cambio significativo, lo que podría afectar la capacidad del modelo para generalizar bien a nuevos datos.


- PSI < 0.1: Generalmente se considera que la característica es estable.
-  PSI entre 0.1 y 0.25: Sugiere cierta inestabilidad, lo que podría indicar la necesidad de investigar más a fondo la distribución de esa característica.
- PSI > 0.25: Indica una inestabilidad significativa, lo que podría afectar el rendimiento del modelo en datos nuevos.

En este  caso, todos los valores de PSI están por debajo de 0.01, lo que indica una estabilidad muy alta para todas las características entre los conjuntos de entrenamiento y prueba. Esto sugiere que las distribuciones de estas características han cambiado muy poco entre los dos conjuntos de datos, lo cual es positivo para la capacidad de mi modelo de generalizar bien a datos no vistos. Es menos probable que mi modelo experimente una degradación en el rendimiento debido a cambios en la distribución de las características.

## 6. Validación Cruzada:

La validación cruzada proporciona una evaluación más robusta del modelo al dividir el conjunto de datos en múltiples particiones. Aquí aplicaremos la validación cruzada usando el modelo de regresión lineal.

In [ ]:
from sklearn.model_selection import cross_val_score

# Crear el modelo
modelo = LinearRegression()

# Aplicar validación cruzada
scores = cross_val_score(modelo, X, y, cv=5, scoring='neg_mean_absolute_error')

# Convertir los scores a valores positivos y calcular la media
mae_scores = -scores
print(f"MAE promedio en validación cruzada: {mae_scores.mean():.2f}")


MAE promedio en validación cruzada: 0.55


1: Ejercicio: Entrena un modelo de regresión con un algoritmo diferente (por ejemplo, Ridge, Lasso, DecisionTreeRegressor o RandomForestRegressor) y compara sus resultados con el modelo de regresión lineal en términos de MAE y MAPE.

2: Ejercicio: Realiza una validación cruzada para el nuevo modelo que entrenaste en el Ejercicio anterior. ¿Cómo cambian las métricas de error?

3:  Ejercicio: Analiza la importancia de las características en el modelo de regresión que elegiste. ¿Qué características tienen más impacto en la predicción de MEDV?

1: Aplicaremos un Random Forest

In [ ]:

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

In [ ]:
# Entrenar el modelo de RandomForestRegressor
#n_estimators= 100 por defecto
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train_scaled, y_train)
y_pred_rf = rf_model.predict(X_test_scaled)

In [ ]:
# Calcular MAE y MAPE para el modelo de RandomForestRegressor
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mape_rf = mean_absolute_percentage_error(y_test, y_pred_rf) * 100
print(f"RandomForestRegressor - MAE: {mae_rf:.2f}, MAPE: {mape_rf:.2f}%")

RandomForestRegressor - MAE: 0.33, MAPE: 18.91%


Comparando los resultados entre el modelo de regresión lineal y el modelo de RandomForestRegressor, se observa que el RandomForestRegressor tiene un rendimiento significativamente mejor. El MAE del modelo de RandomForest es de 0.33, lo que indica un error absoluto medio más bajo en comparación con el MAE de 0.53 de la regresión lineal. Esto significa que, en promedio, las predicciones del RandomForest están más cerca de los valores reales. Además, el MAPE del RandomForest es de 18.91%, considerablemente menor que el 31.95% de la regresión lineal, lo que sugiere que el modelo de RandomForest tiene un error porcentual relativo menor y es más preciso en sus predicciones. Estos resultados indican que el RandomForestRegressor maneja mejor la complejidad y las relaciones no lineales en los datos, logrando una mayor precisión en comparación con la regresión lineal.

2: Validacion cruzada

In [ ]:
from sklearn.model_selection import cross_val_score

# Crear el modelo
rf_model1 = RandomForestRegressor(random_state=42)

# Aplicar validación cruzada
scores = cross_val_score(rf_model1, X, y, cv=5, scoring='neg_mean_absolute_error')

# Convertir los scores a valores positivos y calcular la media
mae_scores = -scores
print(f"MAE promedio en validación cruzada: {mae_scores.mean():.2f}")

MAE promedio en validación cruzada: 0.46




El incremento del MAE de 0.33 en el conjunto de prueba a 0.46 en la validación cruzada sugiere que el modelo de RandomForestRegressor podría estar ligeramente ajustado a los datos específicos de la partición de prueba original, lo que indica un posible sobreajuste. La validación cruzada proporciona una evaluación más robusta y generalizable del rendimiento del modelo, ya que promedia los errores a través de múltiples particiones del conjunto de datos. Aunque el modelo sigue siendo fuerte, el aumento en el MAE promedio destaca la importancia de la validación cruzada para obtener una medida más realista del desempeño del modelo en situaciones del mundo real, donde la variabilidad de los datos es mayor. Esto nos recuerda que un rendimiento excelente en un solo conjunto de prueba puede no traducirse necesariamente en un rendimiento igualmente fuerte en otras muestras o en un entorno de producción.

3:

In [ ]:

importances = rf_model.feature_importances_
features_importances = pd.Series(importances, index=X.columns)
sorted_importances = features_importances.sort_values(ascending=False)
importances1 = pd.DataFrame({
    "Característica": sorted_importances.index,
    "Importancia": sorted_importances.values
})
print(importances1)


  Característica  Importancia
0         MedInc     0.524871
1       AveOccup     0.138443
2       Latitude     0.088936
3      Longitude     0.088629
4       HouseAge     0.054593
5       AveRooms     0.044272
6     Population     0.030650
7      AveBedrms     0.029606


El análisis de la importancia de las características en el modelo de RandomForestRegressor revela que MedInc (ingreso medio) es, con diferencia, el predictor más potente del valor medio de las viviendas (MEDV), con una importancia relativa de 0.5249. Esto sugiere que las variaciones en el ingreso medio explican la mayor parte de la variabilidad en el valor de las viviendas, lo cual es estadísticamente significativo y coherente con la literatura económica.

Las características geográficas (Latitude y Longitude), junto con AveOccup (ocupación promedio), también son importantes, con importancias relativas de aproximadamente 0.0889 y 0.1384 respectivamente, indicando que la ubicación y la densidad de ocupación son factores relevantes en la predicción, pero menos determinantes que el ingreso medio.

Por otro lado, características como HouseAge, AveRooms, Population, y AveBedrms muestran una menor contribución al modelo, con importancias relativas que van desde 0.0296 a 0.0546.